# Notebook To Experiment with Geo Location

Key items explored:
    1) Can we import .shp files from US Govt for use in extracting zip codes
    2) Can we use pyshp and Shapely to load .shp files and correctly identify the zip code based on polygon

In [47]:
%%sh
pip install pyshp
pip install Shapely

  Using cached https://files.pythonhosted.org/packages/81/d1/b8e1b089a8ddd6df74be583d70373eac55c725c6197c115efbd3c3e1509f/Shapely-1.6.4.post2-cp27-cp27mu-manylinux1_x86_64.whl


pillow 4.0.0 requires olefile, which is not installed.
tensorflow 1.7.0 has requirement numpy>=1.13.3, but you'll have numpy 1.11.3 which is incompatible.
pyrax 1.9.8 has requirement python-novaclient==2.27.0, but you'll have python-novaclient 10.2.0 which is incompatible.
tensorboard 1.7.0 has requirement futures>=3.1.1; python_version < "3", but you'll have futures 3.0.5 which is incompatible.
tensorboard 1.7.0 has requirement numpy>=1.12.0, but you'll have numpy 1.11.3 which is incompatible.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/shapefile.py already exists. Specify --upgrade to force replacement.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/shapefile.pyc already exists. Specify --upgrade to force replacement.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/pyshp-2.0.1.dist-info already exists. Specify --upgrade to force replacement.
pillow 4.0.0 requires olefile, which is not installed.
tensorflow 1.7.0 has requirement numpy>=1.13.3, but you'l

In [39]:
import shapefile
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
from shapely.geometry import geo

In [1]:
df = spark.read.csv('/data/al/*', header=True).persist()

In [4]:
# Read in shapefile from location in HDFS
shp = shapefile.Reader('/home/wce/clsadmin/data/shapes/cb_2017_us_zcta510_500k.shp') 

# Manual testing of idea

In [10]:
point = (-69.8007998,44.6594852) # an x,y tuple

all_shapes = shp.shapes() # get all the polygons
all_records = shp.records()
for i in range(0,len(all_shapes)):
    boundary = all_shapes[i] # get a boundary polygon
    if Point(point).within(shape(boundary)): # make a point and see if it's in the polygon
        name = all_records[i][2] # get the second field of the corresponding record
        print ("The point is in", name )

('The point is in', u'04978')


# Scaling 

In [ ]:
# Create a udf which takes in a long/lat and loops through all the .shp files until correct one is found, and returns zip code
def my_udf(row):
    import shapefile
    from shapely.geometry import Point # Point class
    from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
    from shapely.geometry import geo
    import shapely.geometry
    shp = shapefile.Reader('/home/wce/clsadmin/data/shapes/cb_2017_us_zcta510_500k.shp') #open the shapefile
    all_shapes = shp.shapes() # get all the polygons
    all_records = shp.records()
    if row.POSTCODE is None:
        for i in range(0,len(all_shapes)):
            boundary = all_shapes[i] # get a boundary polygon
            if Point(point).within(shape(boundary)): # make a point and see if it's in the polygon
                name = all_records[i][2] # get the second field of the corresponding record
                return name
        return None
    else:
        return row.POSTCODE

udf_calc_zip = udf(my_udf, StringType())

In [165]:
df.withColumn('infer_zip', udf_calc_zip(struct(['LON','LAT','POSTCODE']))).filter(df.POSTCODE.isNull()).show()

# Errors with Above

Unfortunately we were unable to get the above working at scale as we received errors related to our packages.  After research we believe this stems from Shapely and pyshp not being available on our compute nodes (we installed on master node).  Despite our best efforts, we could not ascertain a way to install these packages throughout the cluster.  We opted instead of use an "infer_zip" methodology based on nearest distance to the central long/lat of each zip code.